# Projections of 3D atom coordinates with B-factor

This notebook example goes through how the "blurring" of atomic coordinates are described by B-factors and how B-factors are incorporated into projection calculations

In [12]:
import numpy as np
from scipy.stats import norm
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt

from mosaics.utils import histogram_2d_linear_interpolation
from mosaics.utils import histogram_2d_gaussian_interpolation

## Modeling uncertainty in position

Thermal fluctuations and sample motion during imaging introduce uncertainty into the positions of atom nuclei effectively blurring the resulting image of a structure.
This blurring is frequently modeled by representing atom nuclei -- the scattering centers of the electron beam -- an isotropic Gaussian parameterized by the atom's B-factor.
Implementing this blurring computationally can be accomplished by convolving a perfectly resolved point with a Gaussian with variable width.

In 1-dimension, this B-factor blurring qualitatively looks like,

In [ ]:
# Create a 1D array with a single spike at the center
x = np.linspace(-10, 10, num=256)
y = np.zeros(256)
y[128] = 1

# Convolve the spike with a Gaussian
gauss_1 = norm.pdf(x, loc=0, scale=0.5)
gauss_2 = norm.pdf(x, loc=0, scale=2)

y1 = np.convolve(y, gauss_1, mode="same")
y2 = np.convolve(y, gauss_2, mode="same")


plt.plot(x, y, label="Perfectly resolved")
plt.plot(x, y1, label="Low B-factor")
plt.plot(x, y2, label="High B-factor")
plt.xlabel("Position / Angstrom")
plt.ylabel("Scattering Potential / Volts")
plt.legend()
plt.show()

## Calculating projections of 3D atomic coordinates

High-resolution two-dimensional template matching requires projections of a 3D reference template be generated at different orientations.
Projections can be calculated using the [Fourier-slice Theorem](https://en.wikipedia.org/wiki/Projection-slice_theorem), or they can be constructed via a rotation and projection matrix operating on a set of 3D coordinates.

Let $X\in\mathbb{R}^{3\times n}$ be the matrix representing the atomic positions in some template structure.
These coordinates can be rotated by multiplication with rotation matrix, $R\in\text{SO}(3)$, yielding a set of coordinates $X' = RX$ at a new orientation; rotations in MOSAICS are parameterized by the Euler angles $(\phi, \theta, \psi)$ in the ZYZ convention.
Projecting the rotated set of 3D coordinates onto the XY plane is accomplished by multiplying by the orthogonal projection matrix $P_{xy} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 0 \end{bmatrix}$ to obtain a set of 2D coordinates, $h = P_{xy}X'$

All these rotation and projection operations can be chained together computationally with an illustrative example on the protein 1MH1.


In [ ]:
# Load in 3D atomic coordinates from pre-processed numpy file
coords = np.load(
    "/Users/mgiammar/Documents/MOSAICS/data/parsed_1mh1.npy"
)  # TODO: Move this to a remote location
print(coords.shape, coords.dtype)
coords -= np.mean(coords, axis=0)

R = Rotation.from_euler("ZYZ", [0, 15, 42], degrees=True)
P = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0]])

# Rotate and project the coordinates
coords_rot = R.apply(coords)
coords_proj = np.dot(coords_rot, P.T)
coords_proj = coords_proj[:, :2]

# Plot the projected coordinates
plt.figure(figsize=(6, 6))
plt.scatter(coords_proj[:, 0], coords_proj[:, 1], s=8, alpha=0.5)
plt.xlabel("X / Angstrom")
plt.ylabel("Y / Angstrom")
plt.show()

## Converting continuous coordinates to density on a rasterized grid

Since cross-correlation operates on images and no continuous sets of points, there needs to be a method to convert the continuous atom positions into an image defined on a 2D grid.
One option is to find the nearest pixel to a projected point and add the point's value to that pixel (nearest interpolation), but this lacks the fidelity necessary to accurately model small positional changes and the continuous nature of the reference structure.

Another option is to use linear interpolation where the density of a single is distributed to the 4 nearest pixels proportional to the distance to that pixel.
The example below shows the difference between nearest and linear interpolation using the projected 2D atomic coordinates.

In [ ]:
# Creating grid of 2D position coordinates for histogram calculations
x = np.linspace(-32, 32, num=96)
y = np.linspace(-32, 32, num=96)
extent = (x[0], x[-1], y[0], y[-1])

# Calculate the two different projections
proj_direct, _, _ = np.histogram2d(coords_proj[:, 0], coords_proj[:, 1], bins=[x, y])
proj_linterp = histogram_2d_linear_interpolation(coords_proj[:, 0], coords_proj[:, 1], bins=[x, y])

# Plot the two projections side by side
fig, ax = plt.subplots(1, 2, figsize=(9, 4))

ax[0].imshow(proj_direct.T, cmap="viridis", extent=extent, origin="lower")
ax[0].set_title("Nearest Interpolation")
ax[0].set_xlabel("X / Angstrom")
ax[0].set_ylabel("Y / Angstrom")

ax[1].imshow(proj_linterp.T, cmap="viridis", extent=extent, origin="lower")
ax[1].set_title("Linear Interpolation")
ax[1].set_xlabel("X / Angstrom")
ax[1].set_ylabel("Y / Angstrom")

plt.tight_layout()
plt.show()

## Modeling atom B-factor using a Gaussian Kernel

Both nearest and linear interpolation effectively represent points with perfectly known positions; accurate projections of a reference structure require the modeling of positional uncertainties.
Using the isotropic B-factor per atom, we can construct a Gaussian kernel with width parameter $\sigma^2$ based on the B-factor.
Manipulation of the B-factor definition yields an expression for $\sigma^2$,
$$
B = 8\pi^2 \langle u^2 \rangle \implies \sigma^2 = \langle u^2 \rangle = \frac{B}{8\pi^2},
$$
since the displacement vector, $u$, follows a Gaussian distribution.

MOSAICS includes a utility function ``histogram_2d_gaussian_interpolation`` under the ``mosaics.utils`` submodule for creating a 2D histogram (image) using Gaussian interpolation.
The example below shows the difference between the structure projected with linear interpolation and with Gaussian interpolation.

*Note:* MOSAICS only models isotropic B-factors which have constant variance under rotation.
Anisotropic uncertainties require additional calculations to rotate the covariance matrix along with the 3D coordinates.

*TODO:* Include utility function for converting from a set of isotropic atom B-factors (plus other parameters which contribute to uncertainty) and creates an array of variances.


In [ ]:
# Load in the pre-processed atomic B-factors
bfactors = np.load(
    "/Users/mgiammar/Documents/MOSAICS/data/parsed_1mh1_bfactors.npy"
)  # TODO: Move this to a remote location
variances = bfactors / (8 * np.pi**2)
print(bfactors.shape, bfactors.dtype)

# Creating grid of 2D position coordinates for histogram calculations
x = np.linspace(-32, 32, num=96)
y = np.linspace(-32, 32, num=96)
extent = (x[0], x[-1], y[0], y[-1])

# Calculate the two different projections
proj_linterp = histogram_2d_linear_interpolation(
    coords_proj[:, 0], coords_proj[:, 1], bins=[x, y]
)
proj_ginterp = histogram_2d_gaussian_interpolation(
    coords_proj[:, 0], coords_proj[:, 1], sigma=np.sqrt(variances), bins=[x, y]
)

# Plot the two projections side by side
fig, ax = plt.subplots(1, 2, figsize=(9, 4))

ax[0].imshow(proj_linterp.T, cmap="viridis", extent=extent, origin="lower")
ax[0].set_title("Linear Interpolation")
ax[0].set_xlabel("X / Angstrom")
ax[0].set_ylabel("Y / Angstrom")

ax[1].imshow(proj_ginterp.T, cmap="viridis", extent=extent, origin="lower")
ax[1].set_title("Gaussian Interpolation")
ax[1].set_xlabel("X / Angstrom")
ax[1].set_ylabel("Y / Angstrom")

plt.tight_layout()
plt.show()

## How B-factor fits into map resolution

*Clean up derivations. Size of b-factor relates to local map resolution but need to tie into FSC*

## Interactive projections

Try the interactive sliders below to test how scaling the B-factors and changing the orientation affects the projection.

**Currently buggy when built into a static HTML page**

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact


# Reload the processed data
coords = np.load("/Users/mgiammar/Documents/MOSAICS/data/parsed_1mh1.npy")
coords -= np.mean(coords, axis=0)
bfactors = np.load("/Users/mgiammar/Documents/MOSAICS/data/parsed_1mh1_bfactors.npy")


# Interactive function
def update_plot(phi, theta, psi, bf_scale, bf_add):
    bf_transform = bfactors * bf_scale + bf_add
    variances = bf_transform / (8 * np.pi**2)

    r = Rotation.from_euler("ZYZ", [phi, theta, psi], degrees=True)
    coords_rot = r.apply(coords)

    proj_ginterp = histogram_2d_gaussian_interpolation(
        coords_rot[:, 0], coords_rot[:, 1], sigma=np.sqrt(variances), bins=[x, y]
    )

    plt.imshow(
        proj_ginterp,
        extent=extent,
        origin="lower",
        cmap="viridis",
    )
    plt.colorbar(label="Density")
    plt.xlabel("X / Angstrom")
    plt.ylabel("Y / Angstrom")
    plt.show()


# Create sliders
phi_slider = widgets.FloatSlider(min=0, max=360, step=0.1, value=0, description="Phi")
theta_slider = widgets.FloatSlider(min=0, max=180, step=0.1, value=0, description="Theta")
psi_slider = widgets.FloatSlider(min=0, max=360, step=0.1, value=0, description="Psi")
b_factor_scale_slider = widgets.FloatSlider(
    min=0.1, max=4, step=0.1, value=1, description="B scale"
)
b_factor_additive_slider = widgets.FloatSlider(
    min=0, max=10, step=1, value=0, description="B add"
)

# Display interactive plot
interact(
    update_plot,
    phi=phi_slider,
    theta=theta_slider,
    psi=psi_slider,
    bf_scale=b_factor_scale_slider,
    bf_add=b_factor_additive_slider,
)